In [72]:
import boto3
import json, sys
from pprint import pprint
import pandas as pd


In [94]:
sem_plus_sub = '07-css'  # no capital, no underscore
region = 'us-west-2'
bucket_name = sem_plus_sub+'-textract-upload'

n_char_in_A_ans = 250
n_char_in_B_ans = 1250
n_char_in_C_ans = 1250



In [61]:
def questionify(lines):
    # with open('text.txt', 'r') as file:
    lines = lines

    drop_str = ['ATTEMPT ALL QUESTIONS IN BRIEF.', 'ATTEMPT ANY THREE OF THE FOLLOWING:',
                'ATTEMPT ANY ONE PART OF THE FOLLOWING:',
                'ATTEMPT ALL QUESTIONS IN BRIEF:', 'ATTEMPT ANY THREE OF THE FOLLOWING.',
                'ATTEMPT ANY ONE PART OF THE FOLLOWING.',
                'ATTEMPT ALL QUESTIONS IN BRIEF', 'ATTEMPT ANY THREE OF THE FOLLOWING',
                'ATTEMPT ANY ONE PART OF THE FOLLOWING']
    words = ['MARKS', 'CO', 'QUESTION', 'NOTE', 'QNO.', 'QNO']
    question_end = ['.', '?', ',', '!']
    lines1 = []

    QA, QB, QC, question = [], [], [], ''
    section_a, section_b, section_c, q_line_start, q_line_end = 0, 0, 0, 0, 0
    lines_in_q = True

    # lines = file.readlines()
    for line in lines:
        line = line.split('\n')[0].upper()
        if not len(line) < 5:
            lines1.append(line)

    for w in words: drop_str.append(w)

    lines = []
    for x in (line for line in lines1 if line not in drop_str):
        lines.append(x)

    for i, line in enumerate(lines):
        if line == 'SECTION A': section_a, section_b, section_c = 1, 0, 0
        if line == 'SECTION B': section_a, section_b, section_c = 0, 1, 0
        if line == 'SECTION C': section_a, section_b, section_c = 0, 0, 1

        for qe in question_end:
            if line.endswith(qe):
                if lines_in_q: q_line_start = i
                q_line_end = i

                question = lines[q_line_start + 1]
                if q_line_start < q_line_end:
                    for q in range(q_line_end - q_line_start - 1):
                        q_line_start += 1
                        question +=' ' + lines[q_line_start + 1]

                if section_a: QA.append(question)
                if section_b: QB.append(question)
                if section_c: QC.append(question)

                question = ''
                lines_in_q = True
            else:
                if lines_in_q:
                    q_line_start = i
                    lines_in_q = False
    
#     print(QA)  # miss the first question on every section.
#     print(QB)
#     print(QC)  # for section-c question, leave the first question if answer is not matching.
    return QA, QB, QC

In [ ]:
# amazon s3
s3 = boto3.resource('s3')

In [69]:
lines = []
s3_files = []

# amazon textract
textract = boto3.client(service_name='textract', region_name=region)

for s3_file in s3_files:
    try:
        response = textract.detect_document_text(
            Document={
                "S3Object": {
                    "Bucket":bucket_name,
                    "Name":s3_file}})
        
        for item in response["Blocks"]:
            if item["BlockType"] == "LINE":
                lines.append(item["Text"])
    except Exception as e:
        print(e)

qa, qb, qc = questionify(lines)


['SECTION A 2 X 7 - 14 A) ENLIST DIFFERENT TYPES OF SENSORS INCORPORATED IN A ROBOT.', 'B) DEFINE AUTOMATION.', 'C) DIFFERENTIATE BETWEEN AUTOMATION AND ROBOTICS.', 'D) WHAT IS MEANT BY ACCURACY OF A ROBOT?', 'E) WHAT ARE THE CHARACTERISTICS OF A SERVOMOTOR?', 'F) EXPLAIN THE FUNCTIONS OF A HYDRAULIC FLUID G) EXPLAIN THE WORKING OF AN ELECTROMAGNETIC RELAY.']
['SECTION B 7 X 3 = 21 A) DISCUSS IN DETAIL THE INTEGRATION OF MECHANICAT "SYSTEMS WITH ELECTRONICS AND COMPUTER SYSTEM.', 'B) IDENTIFY THE VARIOUS TYPES OF TRANSFER DEY DERIEI USED IN INDUSTRIAL AUTOMATION. EXPLAIN THE WORKING OF ANY TWO TRANSFER DEVICES WITH THE HELP OF NEAT SHETCHES.', 'C) A ROBOT IS TO BE USED FOR SPOT WELDING INT AN AUTOMOTIVE INDUSTRY FOR SPOT WELDING OF CAR BODY SELECT THE ROOT TO BE USED AND EXPLAIN THE GEOMETRY, CONTROL STRATEGY, TYPE OF DRIVE OF THE SELECTED ROBOTS D) ENLIST THE LAWS OF ROBOTICS DISCUSS VARIOUS TYPES OF ROBOTS IN DETAIL ALONG WITH THEIR APPLICATIONS E) EXPLAIN THE VARIOUS DRIVE SYSTEMS F

In [78]:
qa = ['SECTION A 2 X 7 - 14 A) ENLIST DIFFERENT TYPES OF SENSORS INCORPORATED IN A ROBOT.', 'B) DEFINE AUTOMATION.', 'C) DIFFERENTIATE BETWEEN AUTOMATION AND ROBOTICS.', 'D) WHAT IS MEANT BY ACCURACY OF A ROBOT?', 'E) WHAT ARE THE CHARACTERISTICS OF A SERVOMOTOR?', 'F) EXPLAIN THE FUNCTIONS OF A HYDRAULIC FLUID G) EXPLAIN THE WORKING OF AN ELECTROMAGNETIC RELAY.']
qb = ['SECTION B 7 X 3 = 21 A) DISCUSS IN DETAIL THE INTEGRATION OF MECHANICAT "SYSTEMS WITH ELECTRONICS AND COMPUTER SYSTEM.', 'B) IDENTIFY THE VARIOUS TYPES OF TRANSFER DEY DERIEI USED IN INDUSTRIAL AUTOMATION. EXPLAIN THE WORKING OF ANY TWO TRANSFER DEVICES WITH THE HELP OF NEAT SHETCHES.', 'C) A ROBOT IS TO BE USED FOR SPOT WELDING INT AN AUTOMOTIVE INDUSTRY FOR SPOT WELDING OF CAR BODY SELECT THE ROOT TO BE USED AND EXPLAIN THE GEOMETRY, CONTROL STRATEGY, TYPE OF DRIVE OF THE SELECTED ROBOTS D) ENLIST THE LAWS OF ROBOTICS DISCUSS VARIOUS TYPES OF ROBOTS IN DETAIL ALONG WITH THEIR APPLICATIONS E) EXPLAIN THE VARIOUS DRIVE SYSTEMS FOR ROBOT END EFFECTORS. HOW ARE GRIPPERS CLASSIFIED? EXPLAIN ANY ONE OF THEM.']
qc = ['SECTION C 7X1-7 1 A) DISCUSS THE ROLE OF PLC IN AUTOMATIOU AND ROBOTICS. ENLIST AND EXPLAIN VARIOUS ELEMENTS OF AUTOMATION.']

In [75]:
df = pd.read_csv(r'C:\Users\Ramstein\Desktop\textract\qa.csv')
df.head()

,Id,Question1,Question2,Answer
0,1,AVEI:L,AVEI:L 1.1. WHAT IS STEEL STRUCTURE ? WHAT ARE...,A STEEL STRUCTURE : A STEEL STRUCTURE IS AN AS...
1,2,WHAT ARE THE ADVANTAGES AND DISADVANTAGES OF S...,WRITE ADVANTAGES OF STEEL USED AS A STRUCTURAL...,A ADVANTAGES : FOLLOWING ARE THE ADVANTAGES OF...
2,3,WHAT IS STRUCTURAL STEEL AND WHAT ARE ITS PROP...,DESCRIBE SOME OF THE MECHANICAL PROPERTIES OF ...,STEEL : IT IS PROBABLY THE MOST VERSATILE AND ...
3,4,DISCUSS IN BRIEF A GENERAL CHEMICAL COMPOSITIO...,GIVE THE CHEMICAL COMPOSITION OF STRUCTURAL ST...,NaN


In [86]:
def find_duplicate_que(x):
    print(x)
    

In [87]:
df['Question1'].apply(lambda x: find_duplicate_que(x))

AVEI:L
WHAT ARE THE ADVANTAGES AND DISADVANTAGES OF STEEL USED AS A STRUCTURAL MATERIAL ?
WHAT IS STRUCTURAL STEEL AND WHAT ARE ITS PROPERTIES ?
DISCUSS IN BRIEF A GENERAL CHEMICAL COMPOSITION OF STEEL AND EFFECT OF CARBON ON STEEL.


0    None
1    None
2    None
3    None
Name: Question1, dtype: object

In [90]:
list(df['Question1'])

['A STEEL STRUCTURE : A STEEL STRUCTURE IS AN ASSEMBLAGE OF A GROUP OF AND MEMBERS TO TRANSFER (ELEMENTS) THEM SAFELY EXPECTED TO THE TO GROUND. SUSTAIN DEPENDING THEIR SHARE ON OF THE APPLIED ORIENTATION FORCES OF THE MEMBER IN THE STRUCTURE AND ITS STRUCTURAL USE, THE MEMBER IS SUBJECTED TO FORCES, EITHER AXIAL, BENDING OR TORSION, OR COMBINATION OF FORCES. B. REQUIREMENTS : FOLLOWING ARE THE REQUIREMENTS THAT GOVERN THE STRUCTURAL DESIGN 1. IT SHOULD HAVE ADEQUATE STRENGTH. 2. IT SHOULD HAVE ADEQUATE STABILITY AND RIGIDITY. 3. IT SHOULD BE DURABLE. 4. IT SHOULD NOT INTERFERE WITH THE FUNCTIONAL REQUIREMENT. 5. IT SHOULD BE ECONOMICAL. 6. IT SHOULD BE READILY ADAPTABLE TO FUTURE EXTENSION.',
 'A ADVANTAGES : FOLLOWING ARE THE ADVANTAGES OF STEEL USED AS STRUCTURAL MATERIAL 1. STEEL IS THE ULTIMATE RECYCLABLE MATERIAL. 2. THEY CAN BE ERECTED AT A FASTER RATE. 3. ADDITIONS AND ALTERNATIONS CAN BE MADE EASILY TO STEEL STRUCTURES. 4. STRUCTURAL STEELS IS TOUGH I.E., IT HAS BOTH STRENGTH 

In [97]:
df.at[2, 'Answer'][:n_char_in_A_ans]

'STEEL : IT IS PROBABLY THE MOST VERSATILE AND COMMONLY USED STRUCTURAL MATERIAL. THE ESSENTIAL ELEMENTS IN STEEL ARE METALLIC IRON AND THE ELEMENT OF NON-METALLIC CARBON WITH SMALL QUANTITY OF OTHER ELEMENTS SUCH AS SILICON, CHROMIUM AND COPPER. PROP'